In [1]:
import kodsp as dsp
import json
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import chain
from rank_bm25 import BM25Okapi

openai_key = 'YOUR_API_KEY'

Not loading Cohere because it is not installed.
Not loading Cohere because it is not installed.


In [3]:
def convert_data_to_dsp_v1(data_type):
    dsp_examples = []
    with open(f"./KorWikiTQ_ko_{data_type}.json", "r") as json_file:
        korwikitq = json.load(json_file)['data']
    for idx, datum in enumerate(korwikitq):
        dsp_examples.append(dsp.Example(question=datum['QAS']['question'], 
                                        answer=datum['QAS']['answer']))
    return dsp_examples

In [4]:
def convert_list_to_df(table_list):
    table_df = pd.DataFrame(table_list)
    table_df = table_df.rename(columns=table_df.iloc[0])
    table_df = table_df.drop(table_df.index[0])
    table_df.reset_index(drop=True, inplace=True)
    table_df = table_df.astype('str')
    table_df.columns = ["nan" if col == None else col for col in list(table_df.columns)]
    return table_df

In [5]:
def convert_table_to_text(table_lst):
    table_df = convert_list_to_df(table_lst)
    table = {}
    table['columns'] = table_df.columns.tolist()
    table['values'] = table_df.values.tolist()
    return json.dumps(table, ensure_ascii = False)

# LM

In [6]:
lm = dsp.GPT3(model='gpt-3.5-turbo', api_key=openai_key, model_type='chat')

dsp.settings.configure(lm=lm)

# RM
> 빠른 실험 및 쉬운 재현을 위해 bm25를 사용했습니다.

In [7]:
def retrieve_bm25(question, k):
    return [txt_to_json_dict[' '.join(top_n_table)] for top_n_table in bm25.get_top_n(question.split(" "), corpus, n=k)]

### Corpus 구축
> Retrieval에 활용할 Corpus를 준비합니다. 해당 노트북은 DSP의 작동 방식을 보여주기 위한 튜토리얼 노트북이기 때문에 korwikitq dev 데이터에 포함된 표만을 사용하여 아주 작은 표 corpus를 구축했습니다.

In [8]:
with open(f"./KorWikiTQ_ko_dev.json", "r") as json_file:
    korwikitq = json.load(json_file)['data']

corpus = []
txt_to_json_dict = defaultdict(str)

for datum in korwikitq:
    table_txt = " ".join(list(chain(*datum['TBL'])))
    if table_txt not in txt_to_json_dict:
        corpus.append(table_txt.split(" "))
        txt_to_json_dict[table_txt] = convert_table_to_text(datum['TBL'])

In [9]:
# custom 예제를 위한 table을 하나 추가합니다!

dsba_table = [['이름', '연구실', '학기', '생일'],
              ['마민정', 'DSBA', '1', '3월 4일'],
              ['이지윤', 'DSBA', '2', '9월 23일'],
              ['박새란', 'DSBA', '2', '10월 5일'],
              ['오수지', 'DSBA', '4', '12월 1일'],
              ['김선우', 'DSBA', '4', '4월 12일']]

dsba_txt = " ".join(list(chain(*dsba_table)))
corpus.append(dsba_txt.split(" "))
txt_to_json_dict[dsba_txt] = convert_table_to_text(dsba_table)

bm25 = BM25Okapi(corpus)

# Data

In [10]:
train = convert_data_to_dsp_v1(data_type='train')
dev = convert_data_to_dsp_v1(data_type='dev')

In [11]:
# 원하는 task에 맞게 prefix, desc, instructions만 간단하게 수정해주면 끝입니다!

Question = dsp.Type(prefix="질문: ", desc="${표와 관련된 질문}")
Answer = dsp.Type(prefix="정답: ", desc="${표에서 찾아낸 질문에 대한 답}", format=dsp.format_answers)
Table = dsp.Type(
    prefix="표:\n",
    desc="${JSON 형식으로 주어진 표}",
    format=dsp.passages2text
)
Rationale = dsp.Type(
    prefix="근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 표에 근거하여 우리는 질문에 주어진 대상에 대해 다음과 같은 것들을 알 수 있습니다.",
    desc="${표에서 찾을 수 있는 정답에 대한 중요한 단서}"
)

# template for basic prompt
qa_template_v2 = dsp.Template(instructions="JSON 형식으로 된 다음의 표에 근거하여 주어진 질문에 대해 답하세요.",
                              context=Table(),
                              question=Question(),
                              answer=Answer())


# template for CoT prompt
qa_template_v3 = dsp.Template(instructions="JSON 형식으로 된 다음의 표에 근거하여 주어진 질문에 대해 답하세요.",
                              context=Table(),
                              question=Question(),
                              rationale=Rationale(),
                              answer=Answer())

# Retrieve-then-Read

## 1) Basic Prompt ver.

In [12]:
def retrieve_then_read_QA(question: str, template: dsp.Template) -> str:
    demos = dsp.sample(train, k=5)
    tables = retrieve_bm25(question, k=1)
    
    example = dsp.Example(question=question, context=tables, demos=demos)
    example, completions = dsp.generate(template)(example, stage='qa')

    return completions.answer

In [13]:
retrieve_then_read_QA(dev[5].question, qa_template_v2)
lm.inspect_history(n=1)





JSON 형식으로 된 다음의 표에 근거하여 주어진 질문에 대해 답하세요.

---

질문: 농림부 3대 차관보은 누구입니까?
정답: 구윤석(具允錫)

질문: 브느와 델롬이 2005년에 연출한 작품은 뭔가요?
정답: 프로포지션

질문: 아라라트 예레반의 1975-76 시즌 UEFA 컵위너스컵 1라운드 최종 점수가 어떻게 돼?
정답: 10-1

질문: 애니메이션 박앵귀 10화의 부제를 일본어로 뭐라고 하나요?
정답: 絆のゆくえ

질문: 비주얼 스튜디오 6.0의 코드명이 뭐야?
정답: Aspen

---

출력 형식은 다음과 같습니다.

표:
${JSON 형식으로 주어진 표}

질문: ${표와 관련된 질문}

정답: ${표에서 찾아낸 질문에 대한 답}

---

표:
«{"columns": ["연도", "방송사", "제목", "역할", "비고"], "values": [["2009", "SBS", "녹색마차", "서정란", ""], ["2011", "SBS", "신기생뎐", "최영림", ""], ["2011", "MBC", "불굴의 며느리", "조은수", ""], ["2013", "MBC", "구가의 서", "월선", ""], ["2014", "KBS2", "뻐꾸기 둥지", "백준희", ""], ["2015", "KBS1", "징비록", "요도", ""], ["2015", "MBC", "달콤살벌 패밀리", "김은실", ""], ["2016", "MBC", "옥중화", "다정", ""], ["2016", "MBC", "황금주머니", "배민희", ""], ["2018", "SBS", "시크릿 마더", "신세연", ""]]}»

질문: 손승우가 요도 역을 연기한 드라마는 뭔가요?

정답:징비록





In [14]:
retrieve_then_read_QA("DSBA 연구실에서 4학기를 다니고 있으며, 성이 오씨인 학생의 생일은?", qa_template_v2)
lm.inspect_history(n=1)





JSON 형식으로 된 다음의 표에 근거하여 주어진 질문에 대해 답하세요.

---

질문: 농림부 3대 차관보은 누구입니까?
정답: 구윤석(具允錫)

질문: 브느와 델롬이 2005년에 연출한 작품은 뭔가요?
정답: 프로포지션

질문: 아라라트 예레반의 1975-76 시즌 UEFA 컵위너스컵 1라운드 최종 점수가 어떻게 돼?
정답: 10-1

질문: 애니메이션 박앵귀 10화의 부제를 일본어로 뭐라고 하나요?
정답: 絆のゆくえ

질문: 비주얼 스튜디오 6.0의 코드명이 뭐야?
정답: Aspen

---

출력 형식은 다음과 같습니다.

표:
${JSON 형식으로 주어진 표}

질문: ${표와 관련된 질문}

정답: ${표에서 찾아낸 질문에 대한 답}

---

표:
«{"columns": ["이름", "연구실", "학기", "생일"], "values": [["마민정", "DSBA", "1", "3월 4일"], ["이지윤", "DSBA", "2", "9월 23일"], ["박새란", "DSBA", "2", "10월 5일"], ["오수지", "DSBA", "4", "12월 1일"], ["김선우", "DSBA", "4", "4월 12일"]]}»

질문: DSBA 연구실에서 4학기를 다니고 있으며, 성이 오씨인 학생의 생일은?

정답:12월 1일





## 2) CoT Prompt ver.

In [15]:
retrieve_then_read_QA(dev[5].question, qa_template_v3)
lm.inspect_history(n=1)





JSON 형식으로 된 다음의 표에 근거하여 주어진 질문에 대해 답하세요.

---

질문: 농림부 3대 차관보은 누구입니까?
정답: 구윤석(具允錫)

질문: 브느와 델롬이 2005년에 연출한 작품은 뭔가요?
정답: 프로포지션

질문: 아라라트 예레반의 1975-76 시즌 UEFA 컵위너스컵 1라운드 최종 점수가 어떻게 돼?
정답: 10-1

질문: 애니메이션 박앵귀 10화의 부제를 일본어로 뭐라고 하나요?
정답: 絆のゆくえ

질문: 비주얼 스튜디오 6.0의 코드명이 뭐야?
정답: Aspen

---

출력 형식은 다음과 같습니다.

표:
${JSON 형식으로 주어진 표}

질문: ${표와 관련된 질문}

근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 표에 근거하여 우리는 질문에 주어진 대상에 대해 다음과 같은 것들을 알 수 있습니다. ${표에서 찾을 수 있는 정답에 대한 중요한 단서}

정답: ${표에서 찾아낸 질문에 대한 답}

---

표:
«{"columns": ["연도", "방송사", "제목", "역할", "비고"], "values": [["2009", "SBS", "녹색마차", "서정란", ""], ["2011", "SBS", "신기생뎐", "최영림", ""], ["2011", "MBC", "불굴의 며느리", "조은수", ""], ["2013", "MBC", "구가의 서", "월선", ""], ["2014", "KBS2", "뻐꾸기 둥지", "백준희", ""], ["2015", "KBS1", "징비록", "요도", ""], ["2015", "MBC", "달콤살벌 패밀리", "김은실", ""], ["2016", "MBC", "옥중화", "다정", ""], ["2016", "MBC", "황금주머니", "배민희", ""], ["2018", "SBS", "시크릿 마더", "신세연", ""]]}»

질문: 손승우가 요도 역을 연기한 드라마는 뭔가요?

근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 표에 근거하여

In [16]:
# 이와 같이 demonstration에 주어진 것처럼 단답형으로 대답을 하지 않고 서술형으로 잘못 대답하는 경우 demo 개수를 늘리는 걸 추천드립니다
retrieve_then_read_QA("DSBA 연구실에서 4학기를 다니고 있으며, 성이 오씨인 학생의 생일은?", qa_template_v3)
lm.inspect_history(n=1)





JSON 형식으로 된 다음의 표에 근거하여 주어진 질문에 대해 답하세요.

---

질문: 농림부 3대 차관보은 누구입니까?
정답: 구윤석(具允錫)

질문: 브느와 델롬이 2005년에 연출한 작품은 뭔가요?
정답: 프로포지션

질문: 아라라트 예레반의 1975-76 시즌 UEFA 컵위너스컵 1라운드 최종 점수가 어떻게 돼?
정답: 10-1

질문: 애니메이션 박앵귀 10화의 부제를 일본어로 뭐라고 하나요?
정답: 絆のゆくえ

질문: 비주얼 스튜디오 6.0의 코드명이 뭐야?
정답: Aspen

---

출력 형식은 다음과 같습니다.

표:
${JSON 형식으로 주어진 표}

질문: ${표와 관련된 질문}

근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 표에 근거하여 우리는 질문에 주어진 대상에 대해 다음과 같은 것들을 알 수 있습니다. ${표에서 찾을 수 있는 정답에 대한 중요한 단서}

정답: ${표에서 찾아낸 질문에 대한 답}

---

표:
«{"columns": ["이름", "연구실", "학기", "생일"], "values": [["마민정", "DSBA", "1", "3월 4일"], ["이지윤", "DSBA", "2", "9월 23일"], ["박새란", "DSBA", "2", "10월 5일"], ["오수지", "DSBA", "4", "12월 1일"], ["김선우", "DSBA", "4", "4월 12일"]]}»

질문: DSBA 연구실에서 4학기를 다니고 있으며, 성이 오씨인 학생의 생일은?

근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 표에 근거하여 우리는 질문에 주어진 대상에 대해 다음과 같은 것들을 알 수 있습니다.- 연구실이 DSBA인 학생들의 정보
- 학기가 4인 학생들의 정보
- 성이 오씨인 학생들의 정보

정답: DSBA 연구실에서 4학기를 다니고 있으며, 성이 오씨인 학생의 생일은 12월 1일입니다.





In [18]:
retrieve_then_read_QA("DSBA 연구실에서 4학기를 다니고 있으며, 성이 오씨인 학생의 생일은 어떤 계절에 속해?", qa_template_v3)
lm.inspect_history(n=1)





JSON 형식으로 된 다음의 표에 근거하여 주어진 질문에 대해 답하세요.

---

질문: 농림부 3대 차관보은 누구입니까?
정답: 구윤석(具允錫)

질문: 브느와 델롬이 2005년에 연출한 작품은 뭔가요?
정답: 프로포지션

질문: 아라라트 예레반의 1975-76 시즌 UEFA 컵위너스컵 1라운드 최종 점수가 어떻게 돼?
정답: 10-1

질문: 애니메이션 박앵귀 10화의 부제를 일본어로 뭐라고 하나요?
정답: 絆のゆくえ

질문: 비주얼 스튜디오 6.0의 코드명이 뭐야?
정답: Aspen

---

출력 형식은 다음과 같습니다.

표:
${JSON 형식으로 주어진 표}

질문: ${표와 관련된 질문}

근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 표에 근거하여 우리는 질문에 주어진 대상에 대해 다음과 같은 것들을 알 수 있습니다. ${표에서 찾을 수 있는 정답에 대한 중요한 단서}

정답: ${표에서 찾아낸 질문에 대한 답}

---

표:
«{"columns": ["이름", "연구실", "학기", "생일"], "values": [["마민정", "DSBA", "1", "3월 4일"], ["이지윤", "DSBA", "2", "9월 23일"], ["박새란", "DSBA", "2", "10월 5일"], ["오수지", "DSBA", "4", "12월 1일"], ["김선우", "DSBA", "4", "4월 12일"]]}»

질문: DSBA 연구실에서 4학기를 다니고 있으며, 성이 오씨인 학생의 생일은 어떤 계절에 속해?

근거: 정답을 확실히 하기 위해 이 문제를 단계별로 해결해 봅시다. 표에 근거하여 우리는 질문에 주어진 대상에 대해 다음과 같은 것들을 알 수 있습니다.DSBA 연구실에서 4학기를 다니고 있으며, 성이 오씨인 학생은 오수지입니다. 그리고 오수지의 생일은 12월 1일입니다.

정답: 겨울



